In [1]:
import sys
sys.path.append("/agrilearn_app/agrilearn_app/agrilearn/")

In [2]:
"""This script can be used for training a crop classification model"""
import os
import geopandas as gpd
from geopandas import GeoDataFrame
from tqdm import tqdm
from glob import glob
from eolearn.core import EOPatch
from agrilearn.crop_classification.cropclassifier import CropClassifier
from agrilearn.crop_classification.interface import CropClassificationInterface
from agrilearn.mvp.gpkg_utils import concat_geopackages

2024-12-05 15:17:15.852718: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 15:17:15.871402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733411835.895067   31545 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733411835.900963   31545 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 15:17:15.920802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# export TF_USE_LEGACY_KERAS=True
import tensorflow as tf
from tensorflow.python.client import device_lib
# print(tf.config.list_physical_devices('GPU'))
print("Device atual: ", device_lib.list_local_devices())
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Device atual:  [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18421647159556962308
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14514520064
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17003843051090084571
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
xla_global_id: 416903419
]
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1733411838.185921   31545 gpu_device.cc:2022] Created device /device:GPU:0 with 13842 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5


## Define PATHs

In [11]:
EXPERIMENT_NAME = "experiment_02"
GEOPACKAGE_PATH = "/agrilearn_app/datasets/base/geopackage/raw/crop_classification_raw-checked-v2.gpkg"
EOPATCH_FOLDER = f"/agrilearn_app/datasets/eopatchs/modeling/{EXPERIMENT_NAME}"

# Models and checkpoint path
OUTPUT_CHECKPOINT_PATH = f"/agrilearn_app/output/{EXPERIMENT_NAME}/checkpoints"
OUTPUT_MODEL_PATH = f"/agrilearn_app/output/{EXPERIMENT_NAME}/model"
OUTPUT_PREDICTIONS = f"/agrilearn_app/output/{EXPERIMENT_NAME}/predictions"

In [12]:
#os.makedirs(OUTPUT_CHECKPOINT_PATH, exist_ok=True)

In [13]:
eopatch_files = glob(EOPATCH_FOLDER+"/**", recursive=True)

### 1. Read Datasets

Os valores dessa documentação não batem
- https://pages.experian.local/pages/viewpage.action?pageId=1389466162

In [14]:
df = gpd.read_file(GEOPACKAGE_PATH)

In [15]:
df.shape

(38192, 40)

In [19]:
df.groupby(['dataset_part', 'period']).count()

monitoring_class  fonte  cultura_2  state   area  \
dataset_part period                                                        
test         2017/2018                 8      8          0      8      8   
             2018/2019                24     24          0     24     24   
             2019/2020                28     28          0     28     28   
             2020/2021               970    970          0    970    970   
             2021/2022              1015   1015          0   1015   1015   
train        2017/2018               190    190          0    190    190   
             2018/2019               296    296          0    296    296   
             2019/2020               531    531          0    531    531   
             2020/2021             16035  16035          0  16035  16035   
             2021/2022             17126  17126          0  17126  17126   
val          2017/2018                 8      8          0      8      8   
             2018/2019                21     21          0     21     21   
             2019/2020                36     36          0     36     36   
             2020/2021               829    829          0    829    829   
             2021/2022              1075   1075          0   1075   1075   

                         meso  obs_extra  eopath_location  start_season  \
dataset_part period                                                       
test         2017/2018      8          8                8             8   
             2018/2019     24         24               24            24   
             2019/2020     28         28               28            28   
             2020/2021    970        970              970           970   
             2021/2022   1015       1015             1015          1015   
train        2017/2018    190        190              190           190   
             2018/2019    296        296              296           296   
             2019/2020    531        531              531           531   
             2020/2021  16035      16035            16035         16035   
             2021/2022  17126      17126            17126         17126   
val          2017/2018      8          8                8             8   
             2018/2019     21         21               21            21   
             2019/2020     36         36               36            36   
             2020/2021    829        829              829           829   
             2021/2022   1075       1075             1075          1075   

                        end_season  ...  key_bucket  local_eopatch_path  \
dataset_part period                 ...                                   
test         2017/2018           8  ...           8                   8   
             2018/2019          24  ...          24                  24   
             2019/2020          28  ...          28                  28   
             2020/2021         970  ...         970                 970   
             2021/2022        1015  ...        1015                1015   
train        2017/2018         190  ...         190                 190   
             2018/2019         296  ...         296                 296   
             2019/2020         531  ...         531                 531   
             2020/2021       16035  ...       16035               16035   
             2021/2022       17126  ...       17126               17126   
val          2017/2018           8  ...           8                   8   
             2018/2019          21  ...          21                  21   
             2019/2020          36  ...          36                  36   
             2020/2021         829  ...         829                 829   
             2021/2022        1075  ...        1075                1075   

                        total_imagens_do_intervalo  total_de_imagem  \
dataset_part period                                                   
test         2017/2018               

## 2. Separe train, test and val

In [9]:
train_data = df[df['dataset_part']=='train'].reset_index(drop=True).iloc[:1000]
val_data = df[df['dataset_part']=='val'].reset_index(drop=True).iloc[:100]
test_data = df[df['dataset_part']=='test']

In [10]:
print(f"train_shape: {train_data.shape}")
print(f"val_shape: {val_data.shape}")
print(f"test_shape: {test_data.shape}")

train_shape: (1000, 41)
val_shape: (100, 41)
test_shape: (3636, 41)


### 2. Tranning model

Output do treinamento
- label_enc.pkl: Objeto LabelEnconder, responsável pela transformação da variável alvo para uma representação vetorial e vice-versa.
- minmax_scaler.pkl: Objeto TimestepMinMaxScaler, responsável pela normalização dos valores dos pixels de entrada.
- ts_config.pkl: Dicionário contendo as informações do número de timestamps, número de features e número de classes necessárias para a reconstrução do modelo. 
- rnn_model.h5: Objeto contendo os pesos do modelo treinado

In [11]:
def check_path_exists(path):
    checked = os.path.exists(path)
    return checked

In [12]:
df['checked_path'] = df['local_eopatch_path'].apply(check_path_exists)
df['checked_path'].value_counts()

checked_path
False    73468
Name: count, dtype: int64

In [24]:
crop_classifier = CropClassifier(
    eopatch_folder=EOPATCH_FOLDER,
    model_path=OUTPUT_MODEL_PATH,
    checkpoint_path=OUTPUT_CHECKPOINT_PATH,
)

['CORN', 'SOYBEAN', 'COTTON', 'RICE', 'SUGAR_CANE']


In [25]:
# DEGUB crop_classifer.train

eopatch = EOPatch.load(
    os.path.join(
        EOPATCH_FOLDER,
        train_data.monitoring_class.values[0],
        "train",
        train_data.eopath_location.values[0],
        "eopatch_0_col-0_row-0",
    )
)
eopatch

EOPatch(
  data={
    X_DATA: numpy.ndarray(shape=(3, 50, 3, 13), dtype=float64)
  }
  meta_info={
    LABELS: <class 'list'>, length=3
    TIMESTAMP_SAMPLES: [[3, 4, 5], [4, 5, 6], [5, 6, 7]]
  }
  bbox=BBox(((-52.466159663200926, -32.224125), (-52.44674976809438, -32.21560099999999)), crs=CRS('4326'))
  timestamp=[datetime.datetime(2021, 12, 3, 0, 0), ..., datetime.datetime(2022, 2, 25, 0, 0)], length=8
)

In [26]:
from agrilearn.crop_classification import config
train_batch_size=8
val_batch_size=1
feature_num = eopatch.data["X_DATA"].shape[-1]
train_unique_labels = config.TRAINING_LABELS
ts_sample_n=3

In [27]:
model = crop_classifier._build_rnn_model(
    feature_num, ts_sample_n, len(train_unique_labels)
        )

In [28]:
type(model)

agrilearn.crop_classification.RNNModel.GRUModel

In [32]:
print("fit_generator  labels:", train_unique_labels)
"""Generator to load data in batches."""
model._set_optimizer()
model._set_loss()
model._set_label_encoder(train_unique_labels)
callbacks = model._set_callbacks()

fit_generator  labels: ['CORN', 'SOYBEAN', 'COTTON', 'RICE', 'SUGAR_CANE']


2024-11-23 12:26:28.622133: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-11-23 12:26:28.622163: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-11-23 12:26:28.673607: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-11-23 12:26:28.673713: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed


In [ ]:
model.fit_generator

In [ ]:
# crop_classifier.train(
#     train_df=train_gdf,
#     val_df=val_gdf,
#     test_df=test_gdf,
#     train_batch_size=8,
#     val_batch_size=1,
# )

In [35]:
print("Criando o modelo")
model._create_model()

Criando o modelo


<Sequential name=sequential_2, built=True>

In [33]:
crop_classifier.model